In [4]:
#get all the model and data files from S3 ( only once)
#!aws s3 cp s3://ncw210data/NewClusterFiles/ /home/ubuntu/data/modelsanddata/ --recursive

In [5]:
# same cleaning and stemming function used while creating the csv files from xml to be used for query
import nltk
import re
sb_stemmer = nltk.stem.SnowballStemmer('english')

def tokenize_and_stem(text):
    text_stem = ''
    for word in text.lower().split():
        if text_stem:
            text_stem =  text_stem + ' ' + sb_stemmer.stem(word)
        else:
            text_stem = sb_stemmer.stem(word)
    return text_stem


def clean_stem(string):
    stopwords = {'back', 'thru', 'eg', 'hereafter', 'too', 'part', 'which', 'will', 'be', 'thereupon', 'about', 'nevertheless', 'therein', 'through', 'we', 'among', 'in', 'then', 'former', 'via', 'below', 'whereafter', 'due', 'you', 'bill', 'forty', 'few', 'not', 'with', 'rather', 'next', 'nine', 'me', 'its', 'sometime', 'yours', 'who', 'whoever', 'down', 'some', 'such', 'thereafter', 'hasnt', 'fifteen', 'both', 'as', 'ever', 'could', 'find', 'hence', 'something', 'a', 'there', 'mostly', 'whereas', 'many', 'serious', 'can', 'indeed', 'afterwards', 'whenever', 'by', 'becomes', 'may', 'after', 'couldnt', 'seemed', 'anyhow', 'etc', 'might', 'already', 'no', 'please', 'them', 'myself', 'therefore', 'from', 'along', 'ltd', 'against', 'everywhere', 'amoungst', 'because', 'where', 'sixty', 'ie', 'although', 'sincere', 'move', 'seeming', 'or', 'wherever', 'inc', 'whatever', 'into', 'anywhere', 'around', 'nor', 'see', 'several', 'sometimes', 'for', 'interest', 'beyond', 'whether', 'detail', 'describe', 'moreover', 'nobody', 'whereupon', 're', 'without', 'an', 'ours', 'perhaps', 'only', 'five', 'towards', 'keep', 'eleven', 'one', 'other', 'any', 'otherwise', 'except', 'that', 'cannot', 'behind', 'ourselves', 'under', 'within', 'fifty', 'across', 'if', 'thus', 'per', 'wherein', 'here', 'empty', 'co', 'still', 'whole', 'how', 'off', 'to', 'yourself', 'call', 'cry', 'four', 'so', 'she', 'take', 'their', 'been', 'now', 'even', 'mill', 'what', 'another', 'namely', 'always', 'themselves', 'almost', 'six', 'formerly', 'ten', 'found', 'onto', 'yet', 'between', 'give', 'hers', 'herein', 'eight', 'above', 'anyway', 'third', 'himself', 'front', 'over', 'two', 'much', 'latter', 'itself', 'besides', 'those', 'on', 'twenty', 'up', 'us', 'amongst', 'beforehand', 'but', 'most', 'same', 'mine', 'should', 'this', 'full', 'herself', 'her', 'thick', 'con', 'everything', 'is', 'am', 'three', 'throughout', 'again', 'enough', 'your', 'once', 'hereupon', 'become', 'yourselves', 'everyone', 'before', 'i', 'whereby', 'others', 'must', 'seems', 'elsewhere', 'were', 'either', 'would', 'became', 'hundred', 'toward', 'very', 'latterly', 'top', 'often', 'beside', 'cant', 'else', 'the', 'however', 'and', 'somehow', 'him', 'noone', 'somewhere', 'our', 'nothing', 'de', 'fill', 'well', 'it', 'all', 'last', 'do', 'these', 'has', 'upon', 'every', 'side', 'system', 'put', 'thence', 'twelve', 'becoming', 'show', 'un', 'least', 'of', 'have', 'own', 'since', 'though', 'whither', 'out', 'hereby', 'meanwhile', 'none', 'while', 'whom', 'further', 'why', 'made', 'whose', 'my', 'someone', 'they', 'during', 'anyone', 'first', 'go', 'less', 'his', 'anything', 'thereby', 'amount', 'together', 'never', 'was', 'thin', 'also', 'each', 'fire', 'are', 'when', 'alone', 'had', 'until', 'done', 'more', 'at', 'than', 'nowhere', 'seem', 'whence', 'name', 'neither', 'he', 'get', 'being', 'bottom'}
    #strip and change to lower case and replace commas and semi colons with spaces
    stem = string.strip().lower().replace(';', ' ').replace(',', ' ').replace(':', ' ').replace('(',' ').replace(')',' ').replace('#', ' ').replace('.', ' ').strip()
         
    # remove words that only have numbers( second one removes special characters also)
    stem = re.sub(r'\b\d+\b', ' ',stem).strip()
    
    #remove special characters at the end of words
    stem = re.sub(r'([^\w\s]|_)+(?=\s|$)', ' ',stem).strip()
    
    #remove any words that have a number in it ( even if in the middle )
    stem = re.sub(r'\w*\d\w*', ' ',stem).strip()
    
    # remove any words with only one alphabet
    stem = re.sub(r'\b[a-zA-Z]\b', ' ',stem).strip()
    
    # remove stop words and change to lower case
    stem = ' '.join([item for item in (stem.strip().split()) if item not in stopwords])
    
    # get words greater than length 4 and less than 25
    stem = ' '.join([item for item in (re.findall('\w{4,25}', stem))])
    #re.findall('\w{4,25}', stem).join(' ').strip() 
    
     #apply stemming for all words
    stem = tokenize_and_stem(stem)
    return stem

In [6]:
#query for first level cluster and then second level cluster
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import stop_words

import nltk
sb_stemmer = nltk.stem.SnowballStemmer('english')

my_words = ['','()','(),']
my_stop_words = text.ENGLISH_STOP_WORDS.union(my_words)

def tokenize(doc):
    return doc.lower().split(" ")

def query_first_level_cluster(query_string):
    #find first level cluster
    nmf_filename = '/home/ubuntu/data/modelsanddata/nmfmodels/nmf_model_X.sav'
    tfidf_filename = '/home/ubuntu/data/modelsanddata/tfidfmodels/tfidf_model_X.sav'
    # load the model from disk
    loaded_nmf_model = pickle.load(open(nmf_filename, 'rb'))
    loaded_tfidf_model = pickle.load(open(tfidf_filename, 'rb'))

    query_string_pr_stem = clean_stem(query_string)

    query_tfidf = loaded_tfidf_model.transform([query_string_pr_stem])
    nmf_query = loaded_nmf_model.transform(query_tfidf)
    # print(nmf_query)
    # nmf_query.argmax(axis=1)
    return(nmf_query.argmax(axis=1)[0])

def query_second_level_cluster(query_string, first_level_cluster):
    #find second level cluster
    nmf_filename = '/home/ubuntu/data/modelsanddata/nmfmodels/nmf_model_' + str(first_level_cluster) + '.sav'
    tfidf_filename = '/home/ubuntu/data/modelsanddata/tfidfmodels/tfidf_model_' + str(first_level_cluster) + '.sav' 
    # load the model from disk
    loaded_nmf_model = pickle.load(open(nmf_filename, 'rb'))
    loaded_tfidf_model = pickle.load(open(tfidf_filename, 'rb'))
    
    query_string_pr_stem = clean_stem(query_string)
    
    query_tfidf = loaded_tfidf_model.transform([query_string_pr_stem])
    nmf_query = loaded_nmf_model.transform(query_tfidf)
    # print(nmf_query)
    # nmf_query.argmax(axis=1)
    return(nmf_query.argmax(axis=1)[0])


In [7]:
# test a query
#query_string = '3. The process of claim 1 wherein the amount of polycarboxylic acid employed in the initial charge is equal to between 6 to 12 weight percent in excess of stoichiometry. 4. The process of claim 1 wherein the polyhydric alcohol is selected from the group consisting of ethylene glycol, diethylene glycol, 1,4-butane diol and mixtures thereof. 1. In a process for the preparation of a polyester polyol by the esterification of a polycarboxylic acid or anhydride with a polyhydric alcohol the improvement which comprises:a. initially charging to a reactor an amount of a polycarboxylic acid or anhydride equal to between 3 to 16 weight percent in excess of stoichiometry;b. adding to the reactor a polyhydric alcohol and heating the charge to a temperature between 130.degree. to 240.degree. C.;c. removing from the reactor between 90 to 95 weight percent of the water of esterification resulting from step (b);d. charging to the reactor an amount of polyhydric alcohol substantially equivalent in weight to the amount of water of esterification removed from step (c); ande. continuing the esterification reaction until a polyol having an acid number of less than two is obtained.'    
query_string = 'A method for controlling subscriber access in a network capable of establishing connections with multiple services includes receiving a communication from a subscriber using a first communication network coupled to a second communication network, the communication optionally including a domain identifier associated with a service on the second communication network, and authorizing the subscriber to access a service on the second communication network using a virtual circuit. The authorization is based upon a domain configuration override attribute associated with the virtual circuit used to receive the communication from the subscriber. An access server capable of forcing subscribers of a communications system to gain access exclusively to a domain network associated with a virtual circuit includes an authorizer to grant service authorization to the subscribers based upon a virtual circuit used to make a service request, a virtual circuit profile request generator to generate virtual circuit profile requests and a calculator to determine whether the service associated with the virtual circuit matches the service associated with a domain configuration override attribute.'
query_string = "1 . A method for controlling subscriber access in a network capable of establishing connections with a plurality of services, comprising:   receiving a communication from a subscriber using a first communication network coupled to a second communication network, said communication optionally including a domain identifier associated with a service on said second communication network; and    authorizing said subscriber to access a service on said second communication network using one of a plurality of virtual circuits, said authorizing based upon a domain configuration override attribute associated with the virtual circuit used to receive said communication from said subscriber."
first_cluster = query_first_level_cluster(query_string)
print(first_cluster)
second_cluster = query_second_level_cluster(query_string,first_cluster)
print(second_cluster)

7
6


In [8]:
!sed -n '1000'p /home/ubuntu/data/modelsanddata/smalldatafiles/small_cluster_data_7_6.csv

10973550,US,20041025,20050086495,20050421,178,PPP domain name and L2TP tunnel selection configuration override,"A method for controlling subscriber access in a network capable of establishing connections with multiple services includes receiving a communication from a subscriber using a first communication network coupled to a second communication network, the communication optionally including a domain identifier associated with a service on the second communication network, and authorizing the subscriber to access a service on the second communication network using a virtual circuit. The authorization is based upon a domain configuration override attribute associated with the virtual circuit used to receive the communication from the subscriber. An access server capable of forcing subscribers of a communications system to gain access exclusively to a domain network associated with a virtual circuit includes an authorizer to grant service authorization to the subscribers based upon a 